# Entropy Viscosity approximation from Guermond, Nazarov _et al_ 2014

TODO:
- Fix A2 implementation so that it doens't require the same sparsity pattern _and_ storage order as MB?


We will try to apply the maximum principle preserving entropy viscosity method from Guermond et al 2014 (SINUM) to a simple linear advection (diffusion) problem
$$
\frac{\partial u}{\partial t} + \frac{\partial f}{\partial x}  -\frac{\partial}{dx}\left(a\frac{\partial u}{\partial x}\right)=0 \\
f(u) = uv
$$
over the space-time interval $[0,L_x] \times [0,T]$ with boundary and initial conditions
\begin{align*}
u(x,0) &= u^0 \\
u(t) &= u^b, \mbox{ for } x \in \partial \Omega_d = \{0\} \\
\left.\frac{\partial u}{\partial x}\right. n &= 0, \mbox{ for } \in \partial \Omega_o 
\end{align*}



## The standard Galerkin weak form 
Partioning the interval $[0,L_x]$ into $N_n-1$ cells with $N_n$ nodes, we have the usual discrete, Galerkin weak form in the interior
$$
\int_{\Omega}\frac{\partial u_{h}}{\partial t} w_{h,I} dx - \int_{\Omega} f \frac{dw_{h,I}}{dx} dx + \int_{\Omega} a\frac{du_h}{dx}\frac{dw_{h,I}}{dx} dx = 0, \mbox{ for } I =1,\dots,N_n-2
$$
For the left (Dirichlet boundary) we have
\begin{align}
\left[u_{h,0} - u^b(0,t)\right]w_{h,0}(x=0) &= 0 \\
\end{align}
For the right, we have 
\begin{align}
\int_{\Omega}\frac{\partial u_{h}}{\partial t} w_{h,I} dx - \int_{\Omega} f \frac{dw_{h,I}}{dx} dx + f(x=L_x)w_{h,I} = 0, \mbox{ for } I =N_n-1
\end{align}
where we've written the nodal Lagrange test and trial functions as $w_{h,I}, \ I=1,...,N$ 


## viscous stabilization
The approach from Guermond et al assumes first that there is a low-order approximation that is based (I am pretty sure) on the generalization of the Lax viscosity from traditional finite volume approximations. This low-order viscosity preserves the maximum principle for the (scalar) PDE. Guermond then combines the low-order approximation with  a ''higher order'' entropy viscosity. Both viscosities are used with the graph Laplacian rather than the traditional bilinear form for diffusion

The graph Laplacian on element $\Omega_e$ can be written
\begin{equation}
b_{e}(w_{h,j},w_{h,i}) = \left\{
\begin{array}{ll} -\frac{1}{n_e}|\Omega_e|,  & i \ne j, \ i,j \in \mathcal{I}(\Omega_e) \\
 |\Omega_e|, &  i = j, \ i,j \in \mathcal{I}(\Omega_e) \\
   0, & \mbox{ otherwise} 
\end{array}\right.
\end{equation}

rather than the usual 'diffusion' bilinear form
$$
b_{e}(w_{h,j},w_{h,i}) = \int_{\Omega_e}\frac{d w_{h,j}}{d x}\frac{d w_{h,i}}{d x}dx
$$
Here $\mathcal{I}(\Omega_e)$ is the set of indices for shape functions that have non-zero support over $\Omega_e$, and $n_{e}$ is the number of shape functions that have non-zero support on $\Omega_e$ ($n_d+1$ for $P^1$ shape functions on simplices).


## low-order viscosity
The general definition of the low-order viscosity is 
\begin{align}
\nu_{e}^{L,n} &= \max_{i\ne j \in \mathcal{I}(\Omega_e)} \frac{\max\left[0,\int_{S_{ij}}\left(\vec {f^{'}}(u^n_h)\cdot \nabla w_{h,j}\right)w_{h,i}dx\right]}{\sum_{\Omega_{\ell}\subset S_{ij}} -b_{\ell}\left(w_{h,j},w_{h,i}\right)}
\end{align}
Here, we denote the support of shape function $i$ as $S_{i}$, and $S_{ij}$ is the intersection of the support for shape function $i$ and shape function $j$.

If we define the local advection matrix for an integrated by parts approximation as 
$$
C_{e,ij} = \int_{\Omega_e}\frac{d w_{h,i}}{dx}  w_{h,j} dx
$$
In our 1d example, the denominator is just $|\Omega_e|$. For now, I'll interpret the integral as computing the maximum wave speed (magnitude) along the direction $C_{ji}$ for $i\ne j$. That is $C_{01}$ and $C_{10}$, where we use the local numbering $0,1$ for the left and right nodes on $\Omega_e$. These are $-1/2$ and $1/2$, respectively. 

In other words, I'll use the following formula for the low-order viscosity
\begin{align}
\nu_{e}^{L,n} &= \frac{\max\left[0,\max\left(f^{'}(U_0),f^{'}(U_1)\right),-\min\left(f^{'}(U_0),f^{'}(U_1)\right)\right]}{2|\Omega_e|}
\end{align}


Again, following Guermond et al, we will use a forward Euler approximation with mass lumping. This leads to the simple update at time $t^{n+1}$

\begin{align}
U_{L,I}^{n+1} &= U_{I}^{n} - \Delta t^{n}m_{I}^{-1}\sum_{\Omega_e \subset S_{I}}\left[\nu_{e}^{L,n}b_{e}(u^n_h,w_{h,I})
- \int_{\Omega_e}\vec{f}(u^{n}_h) \nabla w_{h,I}dx \right] + \mbox{ BCs }
\end{align}
where $m_I = \int_{S_I}w_{h,I}dx$. Note also that we've integrated the advection term by parts above.

The CFL constraint for the low-order update is 
\begin{align}
\frac{\beta^n \Delta t^n}{h} \le \frac{1}{\lambda \left(1 + \rho^{-1}\right)} 
\end{align}
where $\beta$ is a global bound on the maximum wave speed $\|\vec {f^{'}}\|$, $h$ a global minimum element diameter, $\lambda$ is a measure of the 'positivity' of the shape functions (see eqns 2.3, 2.4 in Guermond et al) that is always 1 if the shape functions are non-negative, and 
$$
\rho = \min_{\Omega e}\frac{1}{n_{e}-1}
$$
We see that in our simple case, the CFL condition is 
\begin{align}
\frac{\beta^n \Delta t^n}{h} \le \frac{1}{2}
\end{align}

We can also write the scheme as a global update
\begin{align}
\mathbf{U}^{n+1}_{L} = \mathbf{U}^{n} - \Delta t^{n}\mathbf{M}^{-1}_{L}\left(\mathbf{B}^n\mathbf{U}^n + \mathbf{K}\mathbf{U}^n - \mathbf{C}\mathbf{f}^n\right)+ \mathbf{d}_{o}^{n+1}
\end{align}
where
\begin{align}
f^n_{J} &= f(u^n_J,x_J,t^n) \\
d^n_{o,I} &= -f^n(u^n_{I},x_I,t^n)(\pm1), \ x_{I} \in \partial \Omega^n_{out} \\ 
K_{IJ} &= \int_{\Omega} a\frac{dw_{h,J}}{dx}\frac{dw_{h,I}}{dx}dx \\
C_{IJ} &= \int_{\Omega}\frac{ d w_{h,I}}{d x}  w_{h,J} dx
\end{align}
and the nonzero terms of the assembled graph Laplacian operator are
\begin{align}
B^n_{II-1} &= -|\Omega_{I-1}|\nu^{n}_{I-1} \\
B^n_{II+1} &= -|\Omega_{I}|\nu^n_{I} \\
B^n_{II}   &= |\Omega_{I-1}|\nu^n_{I-1} + |\Omega_{I}|\nu^n_{I}
\end{align}

We'll also try just enforcing the Dirichlet constraints after the update via (symbolically)
\begin{align}
\mathbf{D}^{n+1} \mathbf{U}^{n+1} &= \mathbf{d}^{n+1}, \mbox{ where } \\
D^{n}_{IJ} &= \delta_{IJ}, \mbox{ for } x_{I} \in \partial \Omega^n_d \\
d^n_{I} &= u^b(x_I,t^n), \ x_{I} \in \partial \Omega^n_d \\
\end{align}


## entropy viscosity stabilization
For the scalar conservation equation
\begin{align}
\frac{\partial u}{\partial t} + \frac{\partial f}{\partial x} =0 \\
\end{align}
we assume we have an entropy, $\eta$, and entropy flux, $\psi$, that satisfy
\begin{align}
\frac{d\psi}{du} &= \frac{d\eta}{du}\frac{df}{du}
\end{align}
In our linear advection example, we can define $\eta=1/2u^2$ and $\psi=vu^2/2$ or set $\eta= 
-\ln(|(u(1.-u)|+10^{-10})$. 

A basic obervation that motivates using the entropy to scale numerical viscosity is the fact that when $u$ is continuously differentiable, $\eta$ and $\psi$ satisfy
\begin{align}
\frac{\partial \eta}{\partial t} + \frac{\partial \psi}{\partial x} &= \frac{d\eta}{du}\frac{\partial u}{\partial t} 
+ \frac{d \psi}{d u}\frac{\partial u}{\partial x} = \frac{d \eta}{d u}\left(\frac{\partial u}{\partial t} +\frac{\partial f}{\partial x}\right) = 0
\end{align}
More generally, if $\eta$ is a convex function and we have a vanishing viscosity solution $u_{\epsilon}$ to
\begin{align}
\frac{\partial u}{\partial t} + \frac{\partial f}{\partial x} =\epsilon\frac{\partial^2u_{\epsilon}}{\partial x^2} \\
\end{align}
that converges (almost everwhere in $x$ and $t$) to $u$, then we can define the inequality
\begin{align}
\frac{\partial \eta}{\partial t} + \frac{\partial \psi}{\partial x} \le 0
\end{align}
In other words if the inequality above holds everywhere for a solution $u$, as it converges it will converge to the vanishing viscosity solution.

At a discontinuity for $u$, the above observations imply that there will be a source term in the entropy equation that force it to be an inequality. More precisely, if there is an isolated discontinuity at $x(t)$ moving with speed $\sigma = dx/dt$ then there will be a jump in $\eta$ and $\psi$ that satisfies
\begin{align}
[\psi] \le [\eta]\sigma
\end{align}

To go about defining the _entropy viscosity_, we first label the entropy residual over $\Omega_e$ as 
\begin{align}
R_{e} &= \left\|\frac{\partial \eta}{\partial t} + \frac{\partial \psi}{\partial x}\right\|_{L^{\infty}(\Omega_e)} \\
&= \left\|\frac{\partial \eta}{\partial t} + f^{'}\frac{\partial \eta}{\partial x}\right\|_{L^{\infty}(\Omega_e)}
\end{align}
Using a forward Euler approximation for the time derivative, we have at time $t^n$
\begin{align}
R^n_{e} &= \left\|\frac{\eta^{n}-\eta^{n-1}}{\Delta t^{n-1}} + f^{'}\frac{\partial \eta}{\partial x}\right\|_{L^{\infty}(\Omega_e)}
\end{align}
I'll go ahead and try to evaluate $R_e$ at the nodes,
\begin{align}
R^n_{e,i} &= \left\|\frac{\eta(U^{n}_{i})-\eta(U^{n-1}_{i})}{\Delta t^{n-1}} + f^{'}(U^n_i)\frac{\partial \eta_e}{\partial x}\right\|_{L^{\infty}(\Omega_e)}, i=0,1
\end{align}
which for our nodal Lagrange approximation is just
\begin{align}
R^n_{e,i} &= \left\|\frac{\eta(U^{n}_{i})-\eta(U^{n-1}_{i})}{\Delta t^{n-1}} + f^{'}(U^n_i)\frac{\eta(U^n_1)-\eta(U^n_0)}{h_e}\right\|_{L^{\infty}(\Omega_e)}
\end{align}

Guermond et al also include an edge stabilization term 
\begin{align}
J^n_F(u_h) &= \left\|\vec {f^{'}}\cdot \vec n\left[\!\left[\partial_n\eta(u_h)\right]\!\right]\right\|_{\;L^{\infty}(F)}
\end{align}
In 1d, I'll start with the interpretation of the normal jump at node I (the right boundary to $\Omega_{I-1}$)
\begin{align}
\partial_n\eta_{I} &= \left.\frac{\partial \eta}{\partial x}\right|_{I} - \left.\frac{\partial \eta}{\partial x}\right|_{I-1} \\
&= \frac{\eta(U_{I+1})-\eta(U_{I})}{h_{e,I}} - \frac{\eta(U_{I})-\eta(U_{I-1})}{h_{e,I-1}}, \; I=1,\dots,N_x-2
\end{align}

The final high-order viscosity is 
\begin{align}
\nu^{H,n}_e &= \min\left(\nu^{L,n}_e, \frac{c_E R^n_e + C_J\max_{F\subset\partial \Omega_e}J^n_{F}}{\left\|\eta(u^n_h)-\bar{\eta}(u^n_{h})\right\|_{L^{\infty}(\Omega)}}\right)
\end{align}

### Approximating the consisten mass matrix update
The tradeoffs of a lumped versus consistent mass matrix are well known. Kuzmin applies flux-corrected transport (FCT) ideas to control the amount of negative diffusion introduced by the off-diagonal terms in the consistent mass matrix. Guermond et al introduce an approximate inverse. Specifically, they define $\mathbf{\mathcal{B}}=(\mathbf{M}^L-\mathbf{M}^C)(\mathbf{M}^L)^{-1}$, where $\mathbf{M}^{L,C}$ are the lumped and consistent mass matrices, respectively. The consistent mass inverse can be approximated formally as
\begin{align}
\left(\mathbf{M}^C\right)^{-1} &= \left(\mathbf{M}^L\right)^{-1}\left(\mathbf{I}+\mathbf{\mathcal{B}} + \mathbf{\mathcal{B}}^2 + \dots\right)
\end{align}

We can then write the global update for the high-order approximation
\begin{align}
\mathbf{U}^{n+1}_{H} = \mathbf{U}^{n} - \Delta t^{n}\mathbf{M}^{-1}_{L}\left(\mathbf{I}+\mathbf{\mathcal{B}}\right)
\left(\mathbf{B}^n\mathbf{U}^n + \mathbf{K}\mathbf{U}^n - \mathbf{C}\mathbf{f}^n\right)+ \mathbf{d}_{o}^{n+1}
\end{align}

We'll also again try just enforcing the Dirichlet constraints after the update via (symbolically)
\begin{align}
\mathbf{D}^{n+1} \mathbf{U}^{n+1} &= \mathbf{d}^{n+1}, \mbox{ where } \\
D^{n}_{IJ} &= \delta_{IJ}, \mbox{ for } x_{I} \in \partial \Omega^n_d \\
d^n_{I} &= u^b(x_I,t^n), \ x_{I} \in \partial \Omega^n_d \\
\end{align}



In [ ]:
%matplotlib inline
import numpy as np
import scipy
from scipy import linalg,sparse
from scipy.sparse import linalg
import matplotlib.pyplot as plt

In [ ]:
import ev_utils as ev

In [ ]:
reload(ev)

### now grab the cython routines from cev_utils
To compile you should be able to type
```
python setuppyx.py build_ext --inplace
```

In [ ]:
import cev_utils as cev

## Define the problem domain and flux functions

In [ ]:
#for collecting solutions and nonlinear portion of the residual
uh_save ={}
visc_save={}
res_ent_save={}
tn_save = {}


In [ ]:
#spatial domain
L=(0,3.)
#parameters
a0 = 0.0
#time domain
t0=0.
T = 2.0
xic_left=0.1; xic_right=0.5
u_left =  1.0
u_midd =  2.0
u_right=  1.0
velocity=1.
def flux(u,x,t):
    return velocity*u.copy()
def dflux(u,x,t):
    return velocity*np.ones(u.shape,'d')
def entropy(u,x,t):
    #return 0.5*u**2
    return -np.log(np.absolute(u*(1.-u))+1.0e-10)


## Initial and boundary conditions

In [ ]:
def uinit(x,t):
    uic = np.zeros(x.shape); uic.fill(u_right)
    x_support = np.logical_and(x < xic_right,x > xic_left)
    uic[x_support]=u_midd
    uic[np.where(x <= xic_left)]=u_left
    return uic
def dirichlet_bc_left(x,t,u=None):
    x_dir = np.array([L[0]],dtype='d')
    dir_values = np.array([u_left],dtype='d')
    indices = [np.where(np.absolute(x-xd) < 1.0e-10)[0] for xd in x_dir]
    return np.array(indices,dtype='i').squeeze(1),dir_values
def dirichlet_bc_none(x,t,u=None):
    return np.array([],dtype='i'),np.array([],dtype='d')
#Note, the pure Neumann bc will take care of itself in the assembly of K in this case
#but need to tell the code to add the advection term as well
def outflow_bc_right(x,t,u=None):
    #x_out = np.array([L[1]],dtype='d')
    #out_normals = np.array([1.0],'d')
    x_out = np.array([L[1]],dtype='d')
    out_normals = np.array([1.0],'d')
    indices = [np.where(np.absolute(x-xo) < 1.0e-10)[0] for xo in x_out]
    return np.array(indices,dtype='i').squeeze(1),out_normals


def outflow_bc_both(x,t,u=None):
    #x_out = np.array([L[1]],dtype='d')
    #out_normals = np.array([1.0],'d')
    x_out = np.array([L[0],L[1]],dtype='d')
    out_normals = np.array([-1.0,1.0],'d')
    indices = [np.where(np.absolute(x-xo) < 1.0e-10)[0] for xo in x_out]
    return np.array(indices,dtype='i').squeeze(1),out_normals

dirichlet_bc=dirichlet_bc_left
outflow_bc = outflow_bc_right

## low-order viscosity
\begin{align}
\nu_{e}^{L,n} &= \frac{\max\left[0,\max\left(f^{'}(U_0),f^{'}(U_1)\right),
                      -\min\left(f^{'}(U_0),f^{'}(U_1)\right)\right]}
                      {2|\Omega_e|}
\end{align}



In [ ]:
def low_viscosity(he,l2g,df_nodes):
    """
    \begin{align}
    \nu_{e}^{L,n} &= \frac{\max\left[0,\max\left(f^{'}(U_0),f^{'}(U_1)\right),
                                      -\min\left(f^{'}(U_0),f^{'}(U_1)\right)\right]}
                           {2|\Omega_e|}
    \end{align}


    """
    weight_a=0.5/he; weight_b=0.5/he
    df_e = df_nodes[l2g] #map flux derivative to element wise numbering
    df_e01 = weight_a*np.amax(df_e,1)
    df_e10 =-weight_b*np.amin(df_e,1)
    nu_e = np.maximum(df_e01,df_e10)
    nu_e = np.maximum(nu_e,0.)
    return nu_e



## Entropy residual, $R_e$
\begin{align}
R^n_{e,i} &= \left\|\frac{\eta(U^{n}_{i})-\eta(U^{n-1}_{i})}{\Delta t^{n-1}} + 
              f^{'}(U^n_i)\frac{\eta(U^n_1)-\eta(U^n_0)}{h_e}\right\|_{L^{\infty}(\Omega_e)}
\end{align}


In [ ]:
def entropy_residual(he,l2g,dt,t,x,un,unm1,entropy,dflux):
    """
    \begin{align}
    R^n_{e,i} &= \left\|\frac{\eta(U^{n}_{i})-\eta(U^{n-1}_{i})}{\Delta t^{n-1}} + 
                  f^{'}(U^n_i)\frac{\eta(U^n_1)-\eta(U^n_0)}{h_e}\right\|_{L^{\infty}(\Omega_e)}
    \end{align}

    """
    eta = entropy(un,x,t); eta_nm1 = entropy(unm1,x,t)
    eta = eta[l2g]; eta_nm1 = eta_nm1[l2g]
    df = dflux(un,x,t)
    df_e  = df[l2g]
    deta_e  = (eta[:,1]-eta[:,0])/he
    R_e = eta-eta_nm1; R_e /= dt
    R_e[:,0] += df_e[:,0]*deta_e; R_e[:,1] += df_e[:,1]*deta_e
 
    return R_e


def entropy_normalization(eta):
    eta_bar = np.mean(eta);
    eta_diff = eta-eta_bar
    return np.max(np.absolute(eta_diff))



## Edge jump, $J_f$
\begin{align}
 \left[\!\left[\partial_n\eta\right]\!\right]_{I} &= 
 \left.\frac{\partial \eta}{\partial x}\right|_{I} - 
 \left.\frac{\partial \eta}{\partial x}\right|_{I-1} \\
             &= \frac{\eta(U_{I+1})-\eta(U_{I})}{h_{e,I}} - 
                \frac{\eta(U_{I})-\eta(U_{I-1})}{h_{e,I-1}}, \; I=1,\dots,N_x-2
\end{align}



In [ ]:
def edge_jump(he,l2g,eta_nodes,df_nodes):
    """
    \begin{align}
     \left[\!\left[\partial_n\eta\right]\!\right]_{I} &= 
     \left.\frac{\partial \eta}{\partial x}\right|_{I} - 
     \left.\frac{\partial \eta}{\partial x}\right|_{I-1} \\
                 &= \frac{\eta(U_{I+1})-\eta(U_{I})}{h_{e,I}} - 
                    \frac{\eta(U_{I})-\eta(U_{I-1})}{h_{e,I-1}}, \; I=1,\dots,N_x-2
    \end{align}

    """
    eta_jump=np.zeros(eta_nodes.shape)
    deta = (eta_nodes[1:]-eta_nodes[0:-1])/he
    eta_jump[1:-1] = deta[1:]-deta[0:-1]
    eta_jump *= df_nodes
    return np.absolute(eta_jump)



## High-order viscosity
\begin{align}
\nu^{H,n}_e &= \min\left(\nu^{L,n}_e, 
                         \frac{c_E R^n_e + 
                         C_J\max_{F\subset\partial \Omega_e}J^n_{F}}
                         {\left\|\eta(u^n_h)-\bar{\eta}(u^n_{h})\right\|_{L^{\infty}(\Omega)}}\right)
\end{align}



In [ ]:
def high_viscosity(nu_low,Re_norm,entropy_norm,Jf,c_e=1.0,cJ=1.0,eps=1.0e-12):
    """
    \begin{align}
    \nu^{H,n}_e &= \min\left(\nu^{L,n}_e, 
                             \frac{c_E R^n_e + 
                             C_J\max_{F\subset\partial \Omega_e}J^n_{F}}
                             {\left\|\eta(u^n_h)-\bar{\eta}(u^n_{h})\right\|_{L^{\infty}(\Omega)}}\right)
    \end{align}
    """
    return np.minimum(nu_low,(c_e*Re_norm + cJ*Jf)/(entropy_norm+eps))


## Define the discretization parameters

Use either fixed time steps or a target Courant number

In [ ]:
#domain discretization and element coefficients
Nn=151
Ndof=Nn
target_cfl = 0.45
#mesh 
nodes = np.linspace(L[0],L[1],Nn)
he = nodes[1:]-nodes[0:-1]
dir_ind,dir_values = dirichlet_bc(nodes,t0)
out_ind,out_normals = outflow_bc(nodes,t0)
#mapping from local to global unknowns
l2g = np.zeros((Nn-1,2),'i'); l2g[:,0]=np.arange(Nn-1); l2g[:,1]=np.arange(1,Nn)

## Quick and dirty construction of global fem operators

In [ ]:
Mc,M,Minv,MinvH,MB,C,K,D = ev.build_linear_operators(Ndof,a0,he,l2g,dir_ind,out_ind,out_normals)


## initial conditions and work arrays

In [ ]:
uh= uinit(nodes,t0)
dt_stable = ev.choose_stable_dt(uh[l2g],dflux,he,nodes,t0)
dt = target_cfl*dt_stable



In [ ]:
dfn=dflux(uh,nodes,t0)
nu_L = low_viscosity(he,l2g,dfn)
eta = entropy(uh,nodes,t0)
R_e = entropy_residual(he,l2g,dt,t0,nodes,uh,uh,entropy,dflux)
Re_norm = np.amax(np.absolute(R_e),1)
eta_diff = entropy_normalization(eta)
eta_jump = edge_jump(he,l2g,eta,dfn)
Jf = np.amax(np.absolute(eta_jump[l2g]),1)
nu_H = high_viscosity(nu_L,Re_norm,eta_diff,Jf)

In [ ]:
#save the solution and residual
uh_save[(Nn,target_cfl)] = {}
visc_save[(Nn,target_cfl,'low')] = {}
visc_save[(Nn,target_cfl,'high')] = {}
tn_save[(Nn,target_cfl)] = {}
res_ent_save[(Nn,target_cfl)] = {}


#
uh_save[(Nn,target_cfl)][0] = uh
tn_save[(Nn,target_cfl)][0] = t0
visc_save[(Nn,target_cfl,'low')][0] = nu_L
visc_save[(Nn,target_cfl,'high')][0] = nu_H
res_ent_save[(Nn,target_cfl)][0] = Re_norm




## Try to recast implementation as a flux difference

In [ ]:
A2tmp = Mc.data.copy()
A2 = MB.copy()
onevec= np.ones((Ndof,),'d')
FCT_Rm= np.ones((Ndof,),'d')
FCT_Rp= np.ones((Ndof,),'d')

nN_local  = l2g.shape[1]
local_row = np.array([i for i in range(nN_local) for j in range(nN_local)],dtype='int')
local_col = np.array([i for j in range(nN_local) for i in range(nN_local)],dtype='int')


In [ ]:
def fe_step_no_limiting(u,t,dt,R_e,eta,first_order=True):
    #need low order solution for viscosity for now
    F,nu_L = ev.low_order_divergence(u,nodes,t,he,l2g,K,C,Minv,out_ind,out_normals,
                                     flux,dflux,low_viscosity,ev.build_graph_laplacian,
                                     include_inverse_mass=True)
    G,nu_H = ev.high_order_divergence(u,nu_L,nodes,t,R_e,eta,he,l2g,K,C,MinvH,out_ind,out_normals,
                                     flux,dflux,low_viscosity,high_viscosity,
                                     edge_jump,entropy_normalization,
                                     ev.build_graph_laplacian,
                                     include_inverse_mass=True)
    #use negative for rhs update
    u1 = u.copy()
    if first_order:
        u1 -= dt*F
    else:
        u1 -= dt*G
    return u1


In [ ]:
def fe_step_limiting(u,t,dt,R_e,eta,use_limiting=True,debug_imp=False):
    """
    one forward euler step with high-order scheme and limiting
    """
    
    F,nu_L = ev.low_order_divergence(u,nodes,t,he,l2g,K,C,Minv,out_ind,out_normals,
                                     flux,dflux,low_viscosity,ev.build_graph_laplacian,
                                     include_inverse_mass=False)
    G,nu_H = ev.high_order_divergence(u,nu_L,nodes,t,R_e,eta,he,l2g,K,C,MinvH,out_ind,out_normals,
                                     flux,dflux,low_viscosity,high_viscosity,
                                     edge_jump,entropy_normalization,
                                     ev.build_graph_laplacian,
                                     include_inverse_mass=False)



    A1 = ev.build_A1_flux_matrix(u,nu_H,nu_L,he,dt)

    A2tmp = cev.build_A2_flux_matrix(dt,G,A2.indptr,A2.indices,MB.data,A2.data)
    A=A1+A2

    U_L = u.copy(); U_L -= dt*Minv.dot(F)

    
    #mwf debug
    A_before=A.data.copy()

    if use_limiting:
        cev.apply_fct(u,U_L,A.indptr,A.indices,M.diagonal(),A.data,
                  FCT_Rm,FCT_Rp)
    #mwf debug
    #import pdb
    #pdb.set_trace()
    b=A.dot(onevec)
    
    #mwf debug
    if debug_imp:
        diff = Minv.dot(b)+MinvH.dot(G)
        idiff= np.argmax(np.absolute(diff))
        BGdiff = MB.dot(G)+A2.dot(onevec)
        ibgdiff=np.argmax(np.absolute(BGdiff))
        A1diff = (G-F) + A1.dot(onevec)#A1.dot(onevec)
        iA1diff=np.argmax(np.absolute(A1diff))

        MBdense=MB.todense()
        A2dense=Mc.copy().todense();A2dense.fill(0.)
        for I in range(uh.shape[0]):
            for J in range(max(0,I-1),min(uh.shape[0]-1,I+2)):
                #print 'setting I={0} J={1}'.format(I,J)
                A2dense[I,J]=-(MBdense[I,J]*G[J]-MBdense[J,I]*G[I])

        BGdiff2 = MB.dot(G)+A2dense.dot(onevec)
        ibgdiff2=np.argmax(np.absolute(BGdiff2))


        if np.absolute(diff[idiff]) > 1.0e-6:
            import pdb
            pdb.set_trace()

            print 'high-order-limiting t={0} new-diff[{1}]= {2}'.format(t,idiff,diff[idiff])
            print 'high-order-limiting t={0} BGdiff[{1}]={2} A2.nnz={3}'.format(t,ibgdiff,
                                                                                BGdiff[ibgdiff],
                                                                                A2.nnz)
            print 'high-order-limiting t={0} BGdiff2[{1}]={2}'.format(t,ibgdiff2,
                                                                        BGdiff2[0,ibgdiff2])

            print 'high-order-limiting t={0} A1diff[{1}]={2}'.format(t,iA1diff,A1diff[iA1diff])
        #end found some difference
    #end debugging
    uH=Minv.dot(b)
    uH+= U_L
    
    return uH



## Time loop

In [ ]:
t = t0; istep=0
uh0= uinit(nodes,t0)
uh=uh0.copy()
##enforce constraints
uh[dir_ind]=dir_values

dt_stable = ev.choose_stable_dt(uh[l2g],dflux,he,nodes,t0)
dt = target_cfl*dt_stable
uhn=uh.copy()

R_e = entropy_residual(he,l2g,dt,t0,nodes,uh,uhn,entropy,dflux)
eta = entropy(uh,nodes,t)

while t < T:
    
    uh,t,R_e,eta = ev.ssp_rk3_ev(uhn,t,dt,fe_step_limiting,R_e,eta,
                                 entropy_residual,he,l2g,nodes,entropy,dflux)
    #update for next time step
    
    #save viscosities for comparison as well
    dfn=dflux(uh,nodes,t0)
    nu_L = low_viscosity(he,l2g,dfn)
    Re_norm = np.amax(np.absolute(R_e),1)
    eta_diff = entropy_normalization(eta)
    eta_jump = edge_jump(he,l2g,eta,dfn)
    Jf = np.amax(np.absolute(eta_jump[l2g]),1)
    nu_H = high_viscosity(nu_L,Re_norm,eta_diff,Jf)


    #cycle solution
    uhn[:] = uh[:]
    dt_stable = ev.choose_stable_dt(uh[l2g],dflux,he,nodes,t)
    dt = target_cfl*dt_stable
    #save for evaluation later
    uh_save[(Nn,target_cfl)][istep+1]=uh.copy()
    visc_save[(Nn,target_cfl,'low')][istep+1]=nu_L.copy()
    visc_save[(Nn,target_cfl,'high')][istep+1]=nu_H.copy()
    res_ent_save[(Nn,target_cfl)][istep+1] = np.amax(np.absolute(R_e),1)
    tn_save[(Nn,target_cfl)][istep+1]=t
    
    istep += 1



In [ ]:
idt = len(uh_save[(Nn,target_cfl)].keys())-1

In [ ]:
idt

In [ ]:
plt.plot(nodes,uh_save[(Nn,target_cfl)][idt-1])
plt.xlabel('x'); plt.ylabel('u')

In [ ]:
plt.plot(0.5*(nodes[0:-1]+nodes[1:]),res_ent_save[(Nn,target_cfl)][3])
plt.xlabel('x'); plt.ylabel('$R_e$')

In [ ]:
save_a_copy=False
if save_a_copy:
    key='high_order_limiting_log_entropy'
    uh_save[(Nn,target_cfl,key)]={}
    for idt in range(nsnap):
        uh_save[(Nn,target_cfl,key)][idt]=uh_save[(Nn,target_cfl)][idt].copy()

In [ ]:
#IPython 4 from ipywidgets import interact
from IPython.html.widgets import interact

nsnap = len(uh_save[(Nn,target_cfl)])
def plot_soln(idt):
    key='high_order'
    plt.plot(nodes,uh_save[(Nn,target_cfl)][idt],'b-')
    #plt.plot(nodes,uh_save[(Nn,target_cfl)][idt],'b-',
    #         nodes,uh_save[(Nn,target_cfl,'high_order')][idt],'g-',
    #         nodes,uh_save[(Nn,target_cfl,'new_high_order')][idt],'r-',
    #         nodes,uh_save[(Nn,target_cfl,'low_order')][idt],'k-')
    #plt.plot(nodes,uh_save[(Nn,target_cfl,key)][idt],'b-',
    #         nodes,uh_save[(Nn,target_cfl)][idt],'r-')


    u_min=min(u_left,u_midd,u_right); u_max=max(u_left,u_midd,u_right)
    plt.ylim([min(1.05*u_min,0.95*u_min),max(1.05*u_max,0.95*u_max)])
    plt.xlabel('x'); plt.ylabel('u')
def plot_entropy_residual(idt):
    plt.plot(0.5*(nodes[0:-1]+nodes[1:]),res_ent_save[(Nn,target_cfl)][idt])
    plt.xlabel('x'); plt.ylabel('$R_e$')    
def plot_viscosities(idt):
    plt.plot(0.5*(nodes[0:-1]+nodes[1:]),visc_save[(Nn,target_cfl,'low')][idt],'b-',
             0.5*(nodes[0:-1]+nodes[1:]),visc_save[(Nn,target_cfl,'high')][idt],'r-')
    plt.xlabel('x'); plt.ylabel('visc.')
def plot_entropies(idt):
    plt.plot(nodes,entropy(uh_save[(Nn,target_cfl)][idt],nodes,tn_save[(Nn,target_cfl)]),'b-')
    plt.xlabel('x'); plt.ylabel('entropy')    

In [ ]:
interact(plot_soln,idt=(0,nsnap-1,1))

In [ ]:
interact(plot_entropy_residual,idt=(0,nsnap-1,1))

In [ ]:
interact(plot_viscosities,idt=(0,nsnap-1,1))

In [ ]:
interact(plot_entropies,idt=(0,nsnap-1,1))

In [ ]:
save_data_file=False
if save_data_file:
    key=key=(51,target_cfl,'high_order')
    np.savetxt('burgers_nn_x_nn{0:d}.dat'.format(key[0]),nodes)
    idt=len(uh_save[key])
    tmp=np.zeros((Nn,idt),'d')
    for i in range(idt):
        tmp[:,i]=uh_save[key][i]
    np.savetxt('burgers_nn{0:d}_cfl{1:3.2f}_{2}.dat'.format(*key),tmp)